In [1]:
import pandas as pd 

lab_a = pd.read_csv('Invitro.csv')
lab_b = pd.read_csv('OpenHealth.csv')
lab_c = pd.read_csv('Invivo.csv')

lab_a['Лаборатория'] = 'Invitro'
lab_b['Лаборатория'] = 'OpenHealth'
lab_c['Лаборатория'] = 'Invivo'

added the column with name of laboratory to dataframes

In [2]:
all_labs = pd.concat([lab_a, lab_b, lab_c], ignore_index=True)

united all labs into one dataframe

In [3]:
all_labs.columns

Index(['Код', 'Наименование', 'Биоматериал', 'Тип результата',
       'Лаборатория исполнения', 'Алматы, СРОК', 'Партнерская цена, в тенге.',
       'Цена для клиентов на сайте checkdoc', 'Лаборатория',
       'Перечень исследований', 'Розница', 'Партнерская', 'Тип контейнера',
       'Срок готовности', 'Цена', 'Цена со скидкой', 'Сроки выполнения'],
      dtype='object')

In [4]:
all_labs['Партнерская цена'] = all_labs['Партнерская'].combine_first(all_labs['Партнерская цена, в тенге.'])
all_labs['Наименование анализа'] = all_labs['Наименование'].combine_first(all_labs['Перечень исследований'])
all_labs['Сроки готовности'] = all_labs['Срок готовности'].combine_first(all_labs['Сроки выполнения'])
all_labs['Сроки готовности'] = all_labs['Сроки готовности'].combine_first(all_labs['Алматы, СРОК'])

all_labs.drop(columns='Партнерская', inplace=True)
all_labs.drop(columns='Партнерская цена, в тенге.', inplace=True)
all_labs.drop(columns='Наименование', inplace=True)
all_labs.drop(columns='Перечень исследований', inplace=True)
all_labs.drop(columns='Срок готовности', inplace=True)
all_labs.drop(columns='Сроки выполнения', inplace=True)
all_labs.drop(columns='Алматы, СРОК', inplace=True)


так как есть две разных колонки "партнерская" и "партнерская цена в тенге", которые хранят значения для разных лабораторий, объединила их в одну колонку - "партнерская цена". Она работает по принципу если есть значение из первой колонки, то оставляю его, если нет, то ставлю то, что во второй. Тоже самое с наименованием и сроком готовности, а ненужные столбцы удаляем

In [5]:
all_labs["Наименование анализа"].value_counts()

Наименование анализа
Мочевина                                                                                                                                                                                                                                                              3
Витамин С (аскорбиновая кислота)                                                                                                                                                                                                                                      3
Тромбиновое время                                                                                                                                                                                                                                                     3
АПТВ (активированное парциальное тромбопластинновое время)                                                                                                                                 

In [6]:
def is_section_header(row):
    name = str(row['Наименование анализа'])
    return (
        pd.notna(name) and
        pd.isna(row['Код']) and
        pd.isna(row.get('Партнерская цена', None)) and
        pd.isna(row.get('Цена', None)) and
        pd.isna(row.get('Цена для клиентов на сайте checkdoc', None))
    )

all_labs['is_header'] = all_labs.apply(is_section_header, axis=1)


так как в изначальном документе в таблицах OpenHealth и Invivo классификация по категориям проводилась буквально перед всеми анализами (капс в наименовании), относящимся к этой категории, сейчас компьютер думает, что название категории это название еще одного анализа. То есть мы написали функцию, которая выбирает столбец Наименование, ищет название написаное капсом и при этом чтобы остальные строки были NaN. Так как мы указали axis=1, то он будет применять эту функцию к каждой строке и искать название секции. Apply() возвращает True, если всё что написано в теле функции окажется правдой 

In [7]:
category_col = []
current_category = None


for is_header, name in zip(all_labs['is_header'], all_labs['Наименование анализа']):
    if is_header:
        current_category = str(name).title() 
        category_col.append(None)  
    else:
        category_col.append(current_category)

all_labs['Категория'] = category_col

мы создаем пустой лист, куда будем складывать значения категорий для каждого из анализов (то есть построчно). Так же ставим значение текущей категории дефолтное - NaN. В цикле берем булевые значения (категория/не категория) и само наименование анализов. Если категория, то меняем значение текущей категории на это название (но уже не капсом), при этом в лист категорий добавляем NaN, потому что потом мы просто уберем эту строку (так как там лежит не наименование анализа и она нам не нужна). Но если это не категория, то тогда мы добавляем в список категорий название текущей категории, чтобы потом у этого анализа была именна эта категория. В конце уже создаем новую колонку категория, которой присваеваем значения списка

In [8]:
all_labs = all_labs[~all_labs['is_header']].copy()
all_labs.drop(columns='is_header', inplace=True)

`~all_labs['is_header']` тоже самое что НЕ КАТЕГОРИИ, то есть в датафрейме all_labs мы оставляем копию дата фрейма all_labs где в колонке "is_header" лежат не категории (то есть чистим от просто названий категорий)

а дальше уже просто удаляем и саму колонку "is_header", потому что она нам не нужна больше

In [9]:
all_labs['Партнерская цена'] = all_labs['Партнерская цена'].combine_first(all_labs['Цена со скидкой'])
all_labs.drop(columns="Цена со скидкой", inplace=True)

узнала, что партнерская цена = цена со скидкой по значению от ментора, так что теперь я добавила и это в парнерскую цену и удалила лищний столбец 

In [10]:
all_labs[all_labs['Партнерская цена'].isna()]

,Код,Биоматериал,Тип результата,Лаборатория исполнения,Цена для клиентов на сайте checkdoc,Лаборатория,Розница,Тип контейнера,Цена,Партнерская цена,Наименование анализа,Сроки готовности,Категория
1852,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,ГЕМОСТАЗИОЛОГИЧЕСКИЕ ИССЛЕДОВАНИЯ,NaN,Гематологические Исследования
1870,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,ОБЩЕКЛИНИЧЕCКИЕ ИCCЛЕДОВАНИЯ,NaN,Гематологические Исследования
1896,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ МОЧИ,NaN,Гематологические Исследования
1918,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ КРОВИ,NaN,Гематологические Исследования
1967,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,Cпецифические белки (Specific proteins),NaN,Гематологические Исследования
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,Посев на грибы рода Candida с определения чувс...,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Микробиологические Исследования
3796,Посев на золотистый стафилококк с определением...,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Микробиологические Исследования
3804,Прочие посевы,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Микробиологические Исследования
4150,Исполнитель,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,"Антинуклеарные антитела IgG (ANA: dsDNA, Nucle...",NaN,Антинуклеарный Фактор На Клеточной Линии Hep-2...


Проверила, что названия категорий все еще остались типа как анализы. Так же в самом экселе клиники invivo оказались какие-то непонятные строчки, которые не несут в себе никакой реальной информации. Эти строчки не имеют никакой информации, там все колонки (кроме наименования и категории) это NaN.  

In [11]:
bad_rows = (
    (all_labs['Партнерская цена'].isna() | (all_labs['Партнерская цена'].astype(str).str.strip() == '')) &
    (all_labs['Цена для клиентов на сайте checkdoc'].isna() | (all_labs['Цена для клиентов на сайте checkdoc'] == 0.0)) &
    (all_labs['Сроки готовности'].isna() | (all_labs['Сроки готовности'].astype(str).str.strip() == ''))&
    (all_labs['Тип результата'].isna() | (all_labs['Тип результата'].astype(str).str.strip() == '')) 
)

all_labs = all_labs[~bad_rows].copy()



я собрала все строки в которых отсуствует вся информация по критерию что они либо содержат везде NaN, либо 0 или пустую строку и удалила эти плохие строки 

.strip() deletes spaces before and after the text

In [12]:
import re
def clean(text):
    return re.sub(r'[^\w\s]', '', str(text.lower()).strip())

all_labs['Имя без символов'] = all_labs['Наименование анализа'].apply(clean)


re – built-in library in Python to determine the pattern and replace/match it 

re.sub(pattern, replacing to what, text) - this is how it works 

r'[^\w\s]'
w - letters, numbers and underscore
s - space 
[^] - not 

so, r'[^\w\s]' means everything but letters, numbers, underscores and spaces 

there are also many other functions for re 

here, in order to make the function work, we need to use .apply()


In [13]:
all_labs['Имя без символов'].value_counts()

Имя без символов
тромбиновое время                                           3
витамин с аскорбиновая кислота                              3
аптв активированное парциальное тромбопластинновое время    3
парвовирус в19 определение днк                              3
фибриноген                                                  3
                                                           ..
ген фибриногена  fbg g455a                                  1
ген ингибитора активатора плазминогена  pai1 5g4g           1
ген метионинсинтазы  mtr a2756g                             1
ген метионинсинтазы редуктазы  mtrr a66g                    1
обследование на антифосфолипидный синдром афс               1
Name: count, Length: 3751, dtype: int64

I've been sitting and looking at my dataset, trying to figure out how to do mapping. So if the names of the same testings are different now, I need to do something so those names would be the same uniform name. For example, "анализ крови", "оак" in different labs should become "ОАК" in the new table. But then I noticed that in the table of analyses for invivo lab, there are: 
- Кровь. Общий анализ (эритроциты, гемоглобин, лейкоциты, тромбоциты, лейкоформула), капиллярная кровь
- Кровь. Общий анализ (ОАК: эритроциты, гемоглобин, лейкоциты, тромбоциты, СОЭ)
- Кровь. Общий анализ крови (ОАК, 34 параметра) 
- Кровь. Развёрнутый общий анализ крови ( эритроциты, гемоглобин, лейкоциты, тромбоциты, СОЭ, ретикулоциты)
- Клинический анализ крови: общий анализ, лейкоформула, СОЭ (с микроскопией мазка крови при наличии патологических сдвигов) (Clinical Blood Analysis: General Blood Analysis, Leucocyte Formula, ESR (with Microscopic Examination of Blood Smear if Presence of Pathologic Changes))
- Общий анализ крови (ОАК) (без лейкоцитарной формулы и СОЭ) (General Blood Analysis, without White Blood Cell (WBC) Count and ESR)
- ОАК+СОЭ
- ОАК

all of these are ОАК, but they are different. Other labs do not have this thing. This is the hard thing of mapping. So now I need to come up with specific names for these things. I've come up with the pattern to unify it. 
ОАК-базовый 
ОАК+СОЭ 
ОАК-лейкоформула
ОАК-развернутый 

In [ ]:
def classify_names(text):
    if "оак" in text or ("общий анализ" and "кровь") in text or "анализ крови" in text:
        if "развёрнутый" in text or "34" in text or "клинический" in text:
            return "ОАК-развёрнутый"
        elif "лейкоформул" in text: 
            return "ОАК-лейкоформула"
        elif "соэ" in text and "без" not in text: 
            return "ОАК+СОЭ"
        else: 
            return "ОАК-базовый"
    elif "соэ" in text: 
        return "СОЭ"
    elif "фибриноген" in text: 
        return "Фибриноген"
    elif "афс" in text: 
        return "АФС"
    elif "парвовирус" in text: 
        return "ПАРВОВИРУС"
    elif "метионинсинтазы":
        return ""